# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [1]:
# Essentia
!pip install essentia
# Freesound-python
!pip install git+https://github.com/mtg/freesound-python.git
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 4.8 MB/s eta 0:00:00
  Cloning https://github.com/mtg/freesound-python.git to /tmp/pip-req-build-mtsluyzr
  Running command git clone --filter=blob:none --quiet https://github.com/mtg/freesound-python.git /tmp/pip-req-build-mtsluyzr
  Resolved https://github.com/mtg/freesound-python.git to commit 5be99a3689d17303c01cb122bbb0d5a96eba04f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for freesound-python: filename=freesound_python-1.1-py3-none-any.whl size=6714 sha256=73d98a461206e96a43adf935667a7c66d70695b1419e8d84b8459876c683ff4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-_9uczfz0/wheels/2d/14/a0/4f60908e5b801658e71ba0d38958c2e2baef3573ef368db670
Successfully built freesound-python
Mounted at /content/drive


In [5]:
%cd '/content/drive/MyDrive/SMC/AMPLab2324/AMPLAB 2024 Freesound session'

/content/drive/MyDrive/SMC/AMPLab2324/AMPLAB 2024 Freesound session


In [6]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = '36e286ba563287f86eed608c13cacd520ced010e'  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [7]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [8]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'dog bark',
        'filter': None,
        'num_results': 20,
    },
    {
        'query': 'percussion sounds',
        'filter': 'duration:[0 TO 1]',
        'num_results': 20,
    },
    {
        'query': 'vocals',
        'filter': 'duration:[1 TO 2]',
        'num_results': 20,
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')

# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 60 entries! dataframe.csv


,name,username,license,tags,freesound_id,path
0,Jazz the Dog Howl & Bark (053).wav,delphidebrain,https://creativecommons.org/licenses/by-nc/4.0/,"[Barking, Jazz, Barks, Dog, Howling, Wolf, Ani...",236280,files/236280_180659-hq.ogg
1,Dogs bark and howl,WeakWero,http://creativecommons.org/publicdomain/zero/1.0/,"[barking, whine, dog, bark, puppy, dogs, howl]",612856,files/612856_13194336-hq.ogg
2,Animal Dog Bark 01.wav,abhisheky948,http://creativecommons.org/publicdomain/zero/1.0/,"[Bark, Dog, Animal]",625498,files/625498_13366994-hq.ogg
3,dog tripple bark,crazymonke9,https://creativecommons.org/licenses/by/4.0/,"[growling, growl, animals, barking, pet, angry...",418108,files/418108_6078577-hq.ogg
4,"Dog barking, 15m distance",AtoMediaDesign,https://creativecommons.org/licenses/by/4.0/,"[woof, bark, dogs, dog, barking]",649020,files/649020_3571340-hq.ogg
5,Dog Barking Distance - 1.wav,SpaceJoe,http://creativecommons.org/publicdomain/zero/1.0/,"[distance, barking, dog, woof, attack, backgro...",485962,files/485962_6150892-hq.ogg
6,Angry big dog barking - Far [d15].wav,v23,http://creativecommons.org/publicdomain/zero/1.0/,"[barking, far, big, angry, dog, bark]",440865,files/440865_1751509-hq.ogg
7,004 - Dog Barking at Door.wav,TrashCanStudios,https://creativecommons.org/licenses/by/4.0/,"[Foley, Barking, Dog, Canine, Ruff, Woof, Bark...",580213,files/580213_12973965-hq.ogg
8,Dog Bark,aunrea,http://creativecommons.org/publicdomain/zero/1.0/,"[pet, bark, dog]",495658,files/495658_7932944-hq.ogg
9,dogs_barking_mono_4824.wav,Mystikuum,http://creativecommons.org/publicdomain/zero/1.0/,"[barking, dog, canine, woof, animal, bark, dogs]",400916,files/400916_1643758-hq.ogg
